# Probabilistic model of TPTP problems

Author: [Filip Bártek](bartefil@fel.cvut.cz)

## Academic course

This project was completed for the course Probabilistic Models of Uncertainty (XP33PMD).

- Course:
    - Name: Probabilistic Models of Uncertainty
    - Code: XP33PMD
    - Web site: https://cw.fel.cvut.cz/wiki/courses/xp33pmd/start
- Semester: Summer 2020/2021
- University: [Czech Technical University in Prague](https://www.cvut.cz/)

## Task description

[The TPTP (Thousands of Problems for Theorem Provers)](http://www.tptp.org/) is a library of test problems for automated theorem proving systems.
In version 7.4.0, the library contains 23 291 problems.
The problems come from various sources and domains – the distribution is heterogeneous.

Each of the problems in the library is specified in one of the following forms:

1. Clause normal form (CNF) - 8118 problems
2. First-order form (FOF) - 8935 problems
3. Typed first-order form (TFF) - 2234 problems
4. Polymorphic typed higher-order form (THF) - 4004 problems

In this project, I concentrate on the FOF segment of TPTP because it is related to other research projects I work on.

Each FOF problem in TPTP is [annotated with several features](http://www.tptp.org/TPTP/TR/TPTPTR.shtml#HeaderSection), for example:

- [Domain](http://www.tptp.org/TPTP/TR/TPTPTR.shtml#DomainStructure) (for example Set theory or Software verification)
- Source (indicating where the problem was originally published)
- Status (for example Theorem, Satisfiable, or Open)
- Rating (real number in the range 0.0 to 1.0 indicating the difficulty of the problem for the state-of-the-art automated theorem provers)
- Effective order (propositional, effectively propositional, or really first-order)
- Counts of elements (syntactic features):
    - Formulae
    - Atoms
    - Predicates
    - Functors
    - Variables

Besides these, we consider the problem file size as an additional feature.

The main task of this project is to create a probabilistic model of the FOF problems of TPTP v7.4.0.
Each problem is to be represented by a small set of features.
The model is to be an instance of [Bayesian network](https://en.wikipedia.org/wiki/Bayesian_network).
The model is to be used to identify outlier problems and the possible causes of the relative rarity of these problems.

## Implementation

The solution is implemented in Python.
The Bayesian network functionality is implemented in the [pgmpy Python package](https://pgmpy.org/).

### Initialization

In [ ]:
# Install the required Python packages

!pip install daft orderedset pgmpy pickle5

In [ ]:
# Initialize the logger

import logging

# Remove the malfunctioning default handler (apparently installed by Kaggle)
logger = logging.getLogger()
while len(logger.handlers) > 0:
    logger.removeHandler(logger.handlers[0])

logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.INFO)

log = logging.getLogger(__name__)

log.info('Logger initialized.')

In [ ]:
# Initialize the filesystem-based cache

import appdirs
import joblib

memory = joblib.Memory(appdirs.user_cache_dir('strategizer'))

### Data preparation

A table of problems with their features has been prepared in advance and published as the Kaggle dataset [filipbartek/tptp-v740-fof](https://www.kaggle.com/filipbartek/tptp-v740-fof).
We load the table as a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [ ]:
# Load the table of FOF problems with features

import pickle5

filename = '/kaggle/input/tptp-v740-fof/problems.pkl'
with open(filename, 'rb') as f:
    data = pickle5.load(f)

data

We restrict the dataset to 11 features.Some of the features are categorical, while others are numeric.
Both floating point and integer numeric features are included.

In [ ]:
# Filter the problem features

whitelist = ['domain', 'size', 'source', 'rating', 'status', 'effective_order', 'formulae', 'atoms', 'predicates', 'functors', 'variables']
data = data.filter(whitelist)
data.describe(include='all')

Some of the features are categorical, while others are numeric.
Both floating point and integer numeric features are included.

In [ ]:
data.dtypes

We model each problem feature by a discrete random variable.
We discretize the numeric features by binning into 10 intervals.
Problem difficulty rating is binned into intervals of uniform length
and the other numeric features are binned into intervals by quantiles.

In [ ]:
import pandas as pd

def discretize_column(data, column, max_unique=None, bins=10, labels_str=False, cut=None):
    if cut is None:
        cut = lambda x: pd.qcut(x, 10, duplicates='drop')
    x = data[column]
    if isinstance(x.dtype, pd.CategoricalDtype):
        log.info(f'{column}: Skipping because the column is categorical.')
        return
    unique_count = len(x.unique())
    if max_unique is not None and len(x.unique()) <= max_unique:
        log.info(f'{column}: Skipping because the column has {unique_count}<{max_unique} unique values.')
        return
    x = cut(data[column])
    log.info(f'{column}: Discretized {unique_count} unique values to {len(x.unique())} bins.')
    if labels_str:
        x.cat.rename_categories(str, inplace=True)
    data[column] = x

def discretize(data, max_unique=None, labels_str=False, cut=None):
    for column in data.columns:
        discretize_column(data, column, max_unique=max_unique, labels_str=labels_str, cut=cut)

if 'rating' in data:
    discretize_column(data, 'rating', cut=lambda x: pd.cut(x, 10, duplicates='drop'))
discretize(data)
data.describe(include='all')

To ensure that the model does not overfit to the training data,
we split the data into two sets: train set and test set.
The training will be performed on the train set,
while the evaluation will be performed on both the train set and the test set in isolation.

In [ ]:
# Split the problems into train set and test set

from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.5, random_state=0)
datasets = {
    'train': data_train,
    'test': data_test
}

for dataset_name, dataset in datasets.items():
    print(f'Number of {dataset_name} problems: {len(dataset)}')

### Model estimation

We estimate a Bayesian network from the train data.
The estimation proceeds in two steps:

1. Structure estimation
2. Parameter estimation

pgmpy tutorial notebook: [Learning Bayesian Networks from Data](https://github.com/pgmpy/pgmpy_notebook/blob/master/notebooks/9.%20Learning%20Bayesian%20Networks%20from%20Data.ipynb)

#### Structure estimation

The structure estimation is performed by local hill climb search.
The search starts at a completely disconnected network
and iteratively adds and removes edges
so as to maximize the Bayesian Information Criterion (BIC) score (see Section 18.3.5 in [1]).

In [ ]:
# Estimate the structure of a Bayesian network

import pgmpy.estimators

# https://github.com/pgmpy/pgmpy/blob/dev/examples/Structure%20Learning%20in%20Bayesian%20Networks.ipynb
def estimate_structure_hc(data, scoring_method, show_progress=True):
    Estimator = pgmpy.estimators.HillClimbSearch

    est = Estimator(data, scoring_method=scoring_method)

    # The estimated structure is cached because the estimation is computationally expensive.
    estimate_cached = memory.cache(Estimator.estimate, ignore=['show_progress'], verbose=0)
    return estimate_cached(est, scoring_method=scoring_method, show_progress=show_progress)

In [ ]:
dag = estimate_structure_hc(datasets['train'], pgmpy.estimators.BicScore(datasets['train']))

In [ ]:
dag.to_daft(latex=False, node_pos='circular').render(150)

We evaluate the resulting DAG using a number of metrics on both the train and the test data.

In [ ]:
from tqdm.notebook import tqdm

scoring_methods = {
    'K2': pgmpy.estimators.K2Score,
    'BDeu(1)': lambda d: pgmpy.estimators.BDeuScore(d, equivalent_sample_size=1),
    'BDeu(10)': lambda d: pgmpy.estimators.BDeuScore(d, equivalent_sample_size=10),
    'BDeu(n)': lambda d: pgmpy.estimators.BDeuScore(d, equivalent_sample_size=len(d)),
    'BIC': pgmpy.estimators.BicScore
}

model = pgmpy.models.BayesianModel(dag.edges)
for score_name, Score in tqdm(scoring_methods.items(), unit='method', desc='Evaluating the DAG'):
    for dataset_name, dataset in datasets.items():
        score = Score(dataset)
        s = memory.cache(type(score).score, verbose=0)(score, model)
        print(f'{score_name} score on {dataset_name} dataset: {s} ({s / len(dataset)} per problem)')

#### Parameter estimation

Once a locally optimal structure is found,
the Bayesian network parameters (that is the conditional probability distributions)
are estimated by Bayesian parameter estimation using the Bayesian Dirichlet equivalent uniform (BDeu) prior (see section 18.3.6.3 in [1]).

In [ ]:
import warnings

import pgmpy.estimators
import pgmpy.inference
import pgmpy.models
import sklearn

class Model(sklearn.base.BaseEstimator):
    # sklearn estimator wrapper of BayesianModel
    def __init__(self, edges, prior_type='BDeu', equivalent_sample_size=5):
        self.edges = edges
        self.prior_type = prior_type
        self.equivalent_sample_size = equivalent_sample_size
    
    def fit(self, data):
        self.model = pgmpy.models.BayesianModel(self.edges)
        state_names = {column: list(data[column].cat.categories) for column in data}
        self.model.fit(data, state_names=state_names, estimator=pgmpy.estimators.BayesianEstimator, prior_type=self.prior_type, equivalent_sample_size=self.equivalent_sample_size)
    
    def score(self, data):
        return self.log_probability(data).mean()
    
    def log_probability(self, data):
        model = self.model.copy()
        model.remove_nodes_from(model.nodes - data.columns)
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', module='pgmpy')
            return pgmpy.inference.bn_inference.BayesianModelProbability(model).log_probability(data)

BDeu is parameterized by one parameter: the equivalent sample size.
We find the optimum value of the parameter by cross-validation [3].
We optimize the logarithm of the predicted probability of the validation data.

In [ ]:
import scipy

def f(equivalent_sample_size):
    dataset = datasets['train']
    model = Model(dag.edges, equivalent_sample_size=equivalent_sample_size)
    # We invert the score because f will be minimized.
    return -sklearn.model_selection.cross_val_score(model, dataset, error_score='raise').mean()

bounds = (1, len(datasets['train']))
res = scipy.optimize.minimize_scalar(f, bracket=bounds, bounds=bounds, method='bounded', options={'disp': 3})
print(res)
equivalent_sample_size = res.x

Finally, we fit the CPDs on the whole train dataset using the golden equivalent sample size.

In [ ]:
model = Model(dag.edges, equivalent_sample_size=equivalent_sample_size)
model.fit(datasets['train'])

### Inference

#### Problem probability distribution

The model predicts a probability for each problem in the dataset.
We plot the distribution of predicted problem probabilities.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

probs = {}
for name, dataset in datasets.items():
    problem_log_probabilities = model.log_probability(dataset)
    print(f'Mean predicted log probability of {name} problem: {problem_log_probabilities.mean()}')
    probs[name] = problem_log_probabilities

problems = np.concatenate([v.index for v in datasets.values()])
data_annotated = data.copy()
data_annotated.loc[problems, 'log_probability'] = np.concatenate(tuple(probs.values()))
data_annotated.loc[problems, 'probability'] = np.exp(np.concatenate(tuple(probs.values())))
data_annotated.loc[problems, 'dataset'] = np.repeat(tuple(probs.keys()), [len(v) for v in probs.values()])

data_annotated.replace([np.inf, -np.inf], np.nan, inplace=True)

print('Number of problems excluded because they have zero predicted probability:', data_annotated.log_probability.isnull().sum())

plt.figure(figsize=(16, 4))
ax = sns.histplot(data_annotated, x='log_probability', hue='dataset')
ax.set(xlabel='Log probability', ylabel='Problem count')
plt.show()

#### Predicting probability distributions

We begin by visualizing some probability distributions predicted by the model. Inferences are performed by variable elimination.

In [ ]:
import pgmpy.inference

# http://pgmpy.org/inference.html
inference = pgmpy.inference.VariableElimination(model.model)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

def plot_inference(inference, variables, evidence=None, **kwargs):
    q = inference.query(variables=variables, evidence=evidence, show_progress=False)
    plot_distribution(q, variables=variables, evidence=evidence, **kwargs)

def plot_distribution(q, variables=None, evidence=None, conditional=False, title=None, horizontal=False):
    if len(q.scope()) == 1:
        assert variables is None or variables == q.scope()
        if horizontal:
            ax = sns.barplot(y=q.state_names[q.scope()[0]], x=q.values)
            ax.set(title=title, ylabel=q.scope()[0], xlabel=label(q.scope(), evidence))
        else:
            ax = sns.barplot(x=q.state_names[q.scope()[0]], y=q.values)
            ax.set(title=title, xlabel=q.scope()[0], ylabel=label(q.scope(), evidence))
            ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    elif len(q.scope()) == 2:
        if variables is None:
            variables = q.scope()
        if conditional:
            q = q.divide(q.marginalize([variables[0]], inplace=False), inplace=False)
        values = q.values
        if variables != q.scope():
            values = values.transpose()
        ax = sns.heatmap(values,
                         xticklabels=q.state_names[variables[1]],
                         yticklabels=q.state_names[variables[0]])
        ax.set(title=label(variables, evidence, conditional=conditional), xlabel=variables[1], ylabel=variables[0])
    else:
        raise RuntimeError('Too many variables to plot.')

def plot_distributions(distributions, title=None):
    variable = next(iter(distributions.values())).scope()[0]
    assert all(d.scope() == [variable] for d in distributions.values())
    distribution_lengths = [len(d.values) for d in distributions.values()]
    data_dict = {
        'distribution': pd.Series(np.repeat(tuple(distributions.keys()), distribution_lengths), dtype='category'),
        'value': np.concatenate(tuple(d.state_names[d.scope()[0]] for d in distributions.values())),
        'probability': np.concatenate(tuple(d.values for d in distributions.values()))
    }
    df = pd.DataFrame(data_dict)
    ax = sns.catplot(data=df, hue='distribution', x='value', y='probability', kind='bar')
    ax.set(title=title)
    ax.set(xlabel=variable)
    ax.set_xticklabels(rotation=90)

def label(variables, evidence, conditional=False):
    if conditional:
        event = variables[0]
        conditions = variables[1:]
    else:
        event = ','.join(variables)
        conditions = []
    if evidence is not None:
        conditions.extend(f'{k}={v}' for k, v in evidence.items())
    if len(conditions) == 0:
        return f'P({event})'
    else:
        condition = ','.join(conditions)
        return f'P({event}|{condition})'

In [ ]:
plt.figure(figsize=(12, 4))
plot_inference(inference, ['domain'])
plt.show()

plt.figure(figsize=(4, 24))
plot_inference(inference, ['source'], horizontal=True)
plt.show()

plot_inference(inference, ['rating'])
plt.show()

Note that since the model was trained with a positive prior, all the source values, including those that do not appear in the train data, are assigned a non-zero probability.

In [ ]:
import yaml

sources_train = np.unique(datasets['train'].source)
distribution = inference.query(variables=['source'], show_progress=False)
d = {s: float(distribution.get_value(source=s)) for s in data.source.cat.categories if s not in sources_train}
print(f'Sources not included in the train dataset and their probabilities:\n{yaml.dump(d)}')

In [ ]:
plt.figure(figsize=(16, 4))
plot_inference(inference, ['rating', 'domain'])
plt.show()

plt.figure(figsize=(16, 4))
plot_inference(inference, ['rating', 'domain'], conditional=True)
plt.show()

plt.figure(figsize=(16, 4))
plot_inference(inference, ['status', 'domain'], conditional=True)
plt.show()

plt.figure(figsize=(4, 24))
plot_inference(inference, ['source', 'rating'])
plt.show()

plot_inference(inference, ['rating', 'size'])
plt.show()

The filesize of the problem and the number of atoms, predicates, functors, and variables are positively correlated with the number of formulae.

In [ ]:
for variable in ['size', 'atoms', 'predicates', 'functors', 'variables']:
    plot_inference(inference, [variable, 'formulae'])
    plt.show()

Problems with 62 to 96 formulae and 4 to 5 functors seem to form a large atypical cluster.
Plotting their domain distribution shows that they appear in SWC more frequently and HWV less frequently.

In [ ]:
def plot_inference_relative(variable, evidence):
    distribution_0 = inference.query([variable], show_progress=False)
    distribution_1 = inference.query([variable], evidence, show_progress=False)
    log_ratio = np.log(distribution_1.values) - np.log(distribution_0.values)
    state_names = list(inference.state_names_map[variable].values())
    ax = sns.barplot(x=state_names, y=log_ratio)
    evidence_str = ','.join(f'{k}={v}' for k, v in evidence.items())
    ax.set(xlabel=variable, ylabel='Logarithm of probability ratio', title=f'log [P({variable}|{evidence_str}) / P({variable})]')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    plt.show()
    
    plot_distributions({f'P({variable})': distribution_0, f'P({variable}|{evidence_str})': distribution_1})
    plt.show()

plt.figure(figsize=(16, 4))
plot_inference_relative('domain', {'functors': inference.state_names_map['functors'][1], 'formulae': inference.state_names_map['formulae'][5]})

#### Outlier analysis

We analyze the outlier problems by plotting some conditional probability distributions of interest.

We analyze each problem $p$ in isolation.
Let $(F_1=f_1, F_2=f_2, ..., F_n=f_n)$ be the features of problem $p$.
For each feature $F_i$, we calculate the conditional probability that $F_i=f_i$ given the other features of $p$, that is $(F_1=f_1, ..., F_{i-1}=f_{i-1}, F_{i+1}=f_{i+1}, ..., F_n=f_n)$.
We denote this probability as $P(F_i=f_i|\mathrm{problem}=p)$.

If this probability is below the threshold 0.05, we plot the distribution $P(F_i|\mathrm{problem}=p)$ to see which values of $F_i$ the model considers more likely.

Similarly, we plot $P(F_i=f_i,F_j=f_j|\mathrm{problem}=p)$ for each pair of features $F_i, F_j$.

In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from orderedset import OrderedSet

def analyze_problem(problem_name, plot_threshold=0.01):
    problem = data.loc[problem_name]

    def compute_probability(hypothesis, evidence=None, plot_threshold=None):
        if evidence is None:
            evidence_dict = {k: v for k, v in problem.iteritems() if k not in hypothesis}
        else:
            if len(set(hypothesis) & set(evidence)) != 0:
                return np.nan
            evidence_dict = {k: problem[k] for k in evidence}
        distribution = inference.query(variables=hypothesis, evidence=evidence_dict, show_progress=False)
        coordinates = tuple(distribution.name_to_no[v][problem[v]] for v in distribution.variables)
        probability = distribution.values[coordinates]
        if evidence is None and plot_threshold is not None and probability < plot_threshold:
            value = ','.join(f'{column}={problem[column]}' for column in distribution.variables)
            variables_str = ','.join(hypothesis)
            plot_distributions({
                f'P({variables_str}|problem={problem_name})': distribution,
                f'P({variables_str})': inference.query(variables=hypothesis, show_progress=False)
            }, title=f'P({value}|problem={problem_name})={probability}')
            plt.show()
        return probability
    
    n = len(problem)
    single_probabilities = np.fromiter((compute_probability([v], plot_threshold=plot_threshold) for v in problem.index), dtype=float, count=n)
    ticklabels = np.array([f'{k}={v}' for k, v in problem.iteritems()])
    perm = np.argsort(single_probabilities)[::-1]

    ax = sns.barplot(x=ticklabels[perm], y=single_probabilities[perm])
    ax.set(xlabel='X', ylabel=f'P(X|problem={problem_name})')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    plt.show()
    
    pair_probabilities = np.fromiter((compute_probability([h], [v]) for h, v in
                                      itertools.product(problem.index, repeat=2)),
                                     dtype=float,
                                     count=n * n).reshape((n, n))

    ax = sns.heatmap(pair_probabilities[perm, :][:, perm], xticklabels=ticklabels[perm], yticklabels=ticklabels[perm])
    ax.set(title=f'P(Y|X)', xlabel='X', ylabel='Y')
    plt.show()
    
    pair_probabilities = np.fromiter((compute_probability(variables) for variables in
                                      itertools.product(problem.index, repeat=2)),
                                     dtype=float,
                                     count=n * n).reshape((n, n))

    ax = sns.heatmap(pair_probabilities[perm, :][:, perm], xticklabels=ticklabels[perm], yticklabels=ticklabels[perm])
    ax.set(title=f'P(X,Y|problem={problem_name})', xlabel='X', ylabel='Y')
    plt.show()
    
    pair_probabilities = np.fromiter((compute_probability(variables, []) for variables in
                                      itertools.product(problem.index, repeat=2)),
                                     dtype=float,
                                     count=n * n).reshape((n, n))

    ax = sns.heatmap(pair_probabilities[perm, :][:, perm], xticklabels=ticklabels[perm], yticklabels=ticklabels[perm])
    ax.set(title='P(X,Y)', xlabel='X', ylabel='Y')
    plt.show()

First we plot the conditional distributions of the features of the most probable problem in the train set.

In [ ]:
analyze_problem(data_annotated.where(data_annotated.dataset == 'train').nlargest(1, 'log_probability').index[0])

We contrast this with the least probable problem in the train set.

In [ ]:
analyze_problem(data_annotated.where(data_annotated.dataset == 'train').nsmallest(1, 'log_probability').index[0])

Problem SWV483+2 is a software verification problem.
The low number of predicates (2) and functors (2) are unusual in combination with the high number of formulae (75), atoms (213) and variables (2195).
The predicates used in this problem are the equality predicate and a 33-ary predicate that specifies the state of the system modeled by the problem.

Most probable problem in the test set:

In [ ]:
analyze_problem(data_annotated.where(data_annotated.dataset == 'test').nlargest(1, 'log_probability').index[0])

Least probable problem in the test set:

In [ ]:
analyze_problem(data_annotated.where(data_annotated.dataset == 'test').nsmallest(1, 'log_probability').index[0])

## Discussion

The selected typical problems for the train (`LAT355+4`) and the test (`LAT349+4`) dataset
share the same feature representation.
The original cause of the similarity of these problems is most likely due to them coming from the same source (`Urb08`) and domain (`LAT`).

The selected atypical problems (`SWV483+2` and `SWV485+2`) differ in the following features:
`size`, `formulae`, `atoms`, and `variables`.

If we were to search for a probability distribution that generalizes into new problem domains and sources,
it would be useful to form the validation set for hyperparameter tuning
by splitting the train set along the divisions between the domains and sources.

## References

1. Daphne Koller and Nir Friedman. 2009. [Probabilistic Graphical Models: Principles and Techniques](https://mitpress.mit.edu/books/probabilistic-graphical-models) - Adaptive Computation and Machine Learning. The MIT Press.
2. Tomi Silander and Petri Kontkanen and Petri Myllymaki. 2012. [On Sensitivity of the MAP Bayesian Network Structure to the Equivalent Sample Size Parameter](https://arxiv.org/abs/1206.5293).
3. https://en.wikipedia.org/wiki/Cross-validation_(statistics)